----
# Exponential Smoothing
-----

Exponential Smoothing is a forecasting method that applies greater weight to more recent observations while exponentially decreasing the weight of older data points. In this notebook, I will explore and compare the three exponential smoothing methods using the **holtwinters** package in Python:

**1. Simple Exponential Smoothing:**

This method is used for time series data without trend or seasonality. 

It averages past data, giving more weight to recent observations and less to older ones. The smoothing parameter (alpha) adjusts how much emphasis is placed on recent data and is used to set the level (a baseline for predictions, similar to the Naive Forecasting method). 

**2. Holt’s Linear Trend Model (Double Exponential Smoothing):**

This method builds on simple exponential smoothing by adding a trend component (another paramter). This method provides predictions that take into account the level and direction of trend.

**3. Holt-Winters Seasonal Model (Triple Exponential Smoothing):**

This model builds on the Holt's Linear Model by adding a seasonality component. This method provides predictions that take into account the level and direction of trend as well as the observed seasonality in data.

## Set Up
---

In [18]:
import numpy as np
import pandas as pd


import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm

from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt, ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

## Utility Functions
----

In [70]:
def plt_forecast(predictions, fc_method, train, test):
    """
    Description:
    Plots the training data, validation data (actual), and baseline predictions on a single graph.

    Parameters:
    - predictions : A Series containing the predicted values with date indices.
    - fc_method: A string describing the forecasting method used.
    - train: A dataframe with train data.
    - test: A dataframe with test data

    Output:
    The function creates a plot using Plotly to visualise:
        - Training data
        - Validation data 
        - Baseline forecast predictions

    """
    
    # Plot to visualise the training data, test data and baseline prediction
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train.index, y=train['Close'], mode='lines', name="Train"))
    fig.add_trace(go.Scatter(x=test.index, y=test['Close'], mode='lines', name="Validation"))
    fig.add_trace(go.Scatter(x=predictions.index, y=predictions, mode='lines', name="Baseline Forecast"))

    fig.update_layout(
        yaxis_title='Close', 
        xaxis_title='Date',
        title= f'{fc_method}'
    )
    fig.show()

In [71]:
def fcast_evaluation(predicted, actual):
    """
    Description:
    To evaluate forecasting performance using multiple metrics.

    Parameters:
    predicted: Forecasted values.
    actual: Actual observed values.

    Output:
    A dictionary containing the evaluation metrics:
        - 'MSE': Mean Squared Error
        - 'MAE': Mean Absolute Error
        - 'RMSE': Root Mean Squared Error
        - 'MAPE': Mean Absolute Percentage Error
    """

    err= actual - predicted

    # Calculating MSE
    mse = mean_squared_error(actual, predicted)

    # Calculating MAE
    mae = mean_absolute_error(actual, predicted)

    # Calculating RMSE
    rmse = np.sqrt(mse)

    # Calculating MAPE
    abs_percent_err = np.abs(err/actual)
    mape = abs_percent_err.mean() * 100

    return {'MSE': mse,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape
            }

## Data Loading
----

In [72]:
raw_data = pd.read_csv('../../data/msft_cleaned.csv', index_col=0)

In [73]:
raw_data.index = pd.to_datetime(raw_data.index)

In [74]:
raw_data = raw_data.asfreq('D')

In [77]:
raw_data.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2019-09-12,131.667405,132.211832,130.731347,131.352203,2.701000e+07,0.0,0.0
2019-09-13,131.600570,131.868011,130.444847,131.161209,2.336310e+07,0.0,0.0
2019-09-14,130.979714,131.435000,130.155107,130.845998,2.115253e+07,0.0,0.0
2019-09-15,130.358857,131.001988,129.865367,130.530787,1.894197e+07,0.0,0.0
2019-09-16,129.738001,130.568977,129.575628,130.215576,1.673140e+07,0.0,0.0


## Cross Validation Train/Test Split 
----

In [78]:
tscv = TimeSeriesSplit(n_splits=5)

## Simple Exponential Smoothing
---

In [83]:
# Creating a Dataframe to store the results
simple_results_df = pd.DataFrame(columns = ['MSE', 'MAE', 'RMSE', 'MAPE','Smoothing Level'], index = [1,2,3,4,5])

In [84]:
for i, (train_index, test_index) in enumerate(tscv.split(raw_data)):
    
    # Create train/test datraframes using the indexes from tcsv
    train_df = raw_data.iloc[train_index, :]
    test_df = raw_data.iloc[test_index, :]

    simp_exp_smooth = SimpleExpSmoothing(train_df['Close'])
    simp_model = simp_exp_smooth.fit(optimized = True)
    forecasts = simp_model.forecast(test_df.shape[0])

    # Plotting the forecasted values alongside actual data and previous data (training data) 
    print(f"Fold {i+1}:") 
    plt_forecast(forecasts, fc_method='Simple Exponential Smoothing', train=train_df, test= test_df)
    
    results_dict = fcast_evaluation(forecasts.values, test_df['Close'].values)
    results_dict['Smoothing Level'] = simp_model.params['smoothing_level']
    simple_results_df.iloc[i] = results_dict

Fold 1:


Fold 2:


Fold 3:


Fold 4:


Fold 5:


### Evaluation

In [85]:
simple_results_df

,MSE,MAE,RMSE,MAPE,Smoothing Level
1,531.380455,17.384185,23.051691,7.562191,0.745314
2,3182.182868,50.371534,56.41084,16.692884,0.848489
3,686.818708,22.400404,26.207226,9.141777,0.91285
4,6024.527907,69.731175,77.617832,21.822544,0.956605
5,2555.278985,44.23354,50.549767,10.488608,0.970648


-----
**Comment:**

The  smoothing_level parameter across all folds is relatively close to 1, ranging from 0.745 to 0.970. This suggests that the this model is heavily relying on the most recent data points for its forecasts, similar to the naive baseline method in the previous notebook. 

Fold 4 seems to perform the worst, likely due to the high volatility seen in the close price. The model's smoothing parameter is quite high, indicating that it is overly focused on the most recent data points. As a result, it fails to capture the abrupt changes in the data, since it emphasises the immediate data points too much.

Fold 1 performs the best. When examining the data points, they appear relatively stable and in this case, having a high smoothing parameter is beneficial because the recent data is consistent, allowing the model to make more accurate predictions.

Overall, Simple Exponential Smoothing did not provide much improvement in forecasting accuracy over the baseline methods. The model appears to handle stable  data much better than data with high variability (as seen in the difference in perofmrance between Fold 1 and Fold 4). This suggests that more complex methods may be needed to deal with periods of high volatility.


## Holt's Linear Trend Model
----

In [88]:
# Creating a Dataframe to store the results
holt_lin_results_df = pd.DataFrame(columns = ['MSE', 'MAE', 'RMSE', 'MAPE','Smoothing Level', 'Smoothing Trend'], index = [1,2,3,4,5])

In [94]:
for i, (train_index, test_index) in enumerate(tscv.split(raw_data)):
    
    # Create train/test datraframes using the indexes from tcsv
    train_df = raw_data.iloc[train_index, :]
    test_df = raw_data.iloc[test_index, :]

    holt_lin = Holt(train_df['Close'], damped_trend=True)
    holt_lin_model = holt_lin.fit(optimized = True)
    forecast_2 = holt_lin_model.forecast(test_df.shape[0])

    # Plotting the forecasted values alongside actual data and previous data (training data) 
    print(f"Fold {i+1}:") 
    plt_forecast(forecast_2, fc_method='Holt\'s Linear Trend Model', train=train_df, test= test_df)
    
    results_dict = fcast_evaluation(forecast_2.values, test_df['Close'].values)
    results_dict['Smoothing Level'] = holt_lin_model.params['smoothing_level']
    results_dict['Smoothing Trend'] = holt_lin_model.params['smoothing_trend']
    holt_lin_results_df.iloc[i] = results_dict

Fold 1:


Fold 2:


Fold 3:


Fold 4:


Fold 5:


In [95]:
holt_lin_results_df

,MSE,MAE,RMSE,MAPE,Smoothing Level,Smoothing Trend
1,550.982206,17.824157,23.47301,7.759444,0.679274,0.071004
2,3186.331339,50.409494,56.447598,16.705818,0.814849,0.044171
3,674.255274,22.195874,25.966426,9.05267,0.899158,0.0183
4,6024.525483,69.731158,77.617817,21.822538,0.95661,0.0
5,2555.27929,44.233544,50.54977,10.488609,0.970652,0.0


-----
**Comment:**

The smoothing_level (emphasis on recent data points) and smoothing_trend (model's ability to capture trend changes over time) parameters in Holt's Linear Trend method show significant variation across folds. The smoothing level ranges from 0.679 to 0.970, but the smoothing_trend values are very low, especially in Folds 4 and 5 where it is 0.0, indicating hardly any emphasis on the trend.

Fold 4 again has the worst performance with MSE of 6024.53 and MAPE of 21.82%. This is in line with findings from simple exponential method, the high smoothing level and zero trend parameter suggest the model is heavily focused on recent data without accounting for trends and so fails to capture the high volatility in this fold, leading to poor forecasts.

Overall, the model's forecasts are horizontal lines across folds, suggesting that Holt’s method is not effectively capturing trends in the stock data. This could be due to the trend component being too low, indicating that more complex methods might be needed to better capture trends and improve forecasting accuracy.

## Holt Winter's Method
----

In [97]:
# Creating a Dataframe to store the results
holt_win_results_df = pd.DataFrame(columns = ['MSE', 'MAE', 'RMSE', 'MAPE','Smoothing Level', 'Smoothing Trend', 'Smoothing Seasonal'], index = [1,2,3,4,5])

In [100]:
for i, (train_index, test_index) in enumerate(tscv.split(raw_data)):
    
    # Create train/test datraframes using the indexes from tcsv
    train_df = raw_data.iloc[train_index, :]
    test_df = raw_data.iloc[test_index, :]

    exp_smooth = ExponentialSmoothing(train_df['Close'],
                                    trend = 'add',
                                    seasonal= 'add',
                                    seasonal_periods= 7)
    holt_win_model = exp_smooth.fit(optimized=True)
    forecast_3 = holt_win_model.forecast(test_df.shape[0])

    # Plotting the forecasted values alongside actual data and previous data (training data) 
    print(f"Fold {i+1}:") 
    plt_forecast(forecast_3, fc_method='Holt Winter\'s Model', train=train_df, test= test_df)
    
    results_dict = fcast_evaluation(forecast_3.values, test_df['Close'].values)
    results_dict['Smoothing Level'] = holt_win_model.params['smoothing_level']
    results_dict['Smoothing Trend'] = holt_win_model.params['smoothing_trend']
    results_dict['Smoothing Seasonal'] = holt_win_model.params['smoothing_seasonal']
    holt_win_results_df.iloc[i] = results_dict

Fold 1:


Fold 2:


Fold 3:


Fold 4:


Fold 5:


In [101]:
holt_win_results_df

,MSE,MAE,RMSE,MAPE,Smoothing Level,Smoothing Trend,Smoothing Seasonal
1,448.909365,19.113851,21.187481,8.811859,0.731787,0.0,0.0
2,925.709179,25.177928,30.425469,8.307376,0.842768,0.0,0.000001
3,2576.498502,43.323948,50.759221,17.8309,0.911878,0.000001,0.0
4,4011.140248,56.857026,63.333563,17.799596,0.957578,0.0,0.0
5,728.41202,21.613052,26.989109,5.100157,0.968653,0.000005,0.0


----
**Comment:**

The smoothing_level, smoothing_trend and smoothing_seasonal parameters in Holt-Winters’ method show varying values across folds. The smoothing_level ranges from 0.731 to 0.969 and indicates high weight is given to recent data points. The smoothing_trend values are very low or zero, ranging from 0.0 to 0.000005, reflecting minimal emphasis on changes in the trend. The smoothing_seasonal parameter is zero across all folds, suggesting no seasonal component was applied. This could be due to the seasonlity in the data being weak.

Fold 4 performs the worst with MSE of 4011.14 and MAPE of 17.80%. The high smoothing level with zero trend and seasonal parameters indicates the model is overly focused on recent data without accounting for trends or seasonality, and so fails to handle the volatility present in this fold.

Fold 5 shows relatively good performance with MSE of 728.41 and MAPE of 5.10%. The high smoothing level (0.9687) and very low smoothing trend (0.000005) suggest that the model captures recent changes well but still does not address seasonality.

Overall, the Holt-Winters method shows some improvement by capturing trends in forecasts. However, the zero values for the seasonal parameter suggest that the data's seasonality may not be strong enough to be effectively captured by this method. 


## Conclusion
-----

The exponential smoothing methods explored in this notebook did not perform well with the stock data.

Simple Exponential Smoothing, Holt's Linear Trend and Holt-Winters’ methods showed varying results but overall they all struggled in capturing trends and seasonality effectively.

Advanced forecasting models such as ARIMA may be needed in order to better capture the complexities seen in this data.